Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.400377
Minibatch accuracy: 6.2%
Validation accuracy: 12.4%
Minibatch loss at step 50: 1.446011
Minibatch accuracy: 56.2%
Validation accuracy: 67.3%
Minibatch loss at step 100: 0.413850
Minibatch accuracy: 93.8%
Validation accuracy: 75.1%
Minibatch loss at step 150: 0.694721
Minibatch accuracy: 68.8%
Validation accuracy: 76.1%
Minibatch loss at step 200: 0.388732
Minibatch accuracy: 87.5%
Validation accuracy: 78.7%
Minibatch loss at step 250: 0.738386
Minibatch accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 300: 1.036651
Minibatch accuracy: 62.5%
Validation accuracy: 78.0%
Minibatch loss at step 350: 1.019326
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.887737
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 450: 0.427390
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 500: 0.460527
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [10]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [11]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.973063
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.215007
Minibatch accuracy: 18.8%
Validation accuracy: 27.9%
Minibatch loss at step 100: 1.259335
Minibatch accuracy: 56.2%
Validation accuracy: 60.6%
Minibatch loss at step 150: 1.028041
Minibatch accuracy: 68.8%
Validation accuracy: 65.8%
Minibatch loss at step 200: 0.475310
Minibatch accuracy: 87.5%
Validation accuracy: 74.6%
Minibatch loss at step 250: 0.899825
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 300: 0.899422
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 350: 1.201593
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 400: 0.845665
Minibatch accuracy: 75.0%
Validation accuracy: 82.1%
Minibatch loss at step 450: 0.421681
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
Minibatch loss at step 500: 0.668926
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [16]:
train_dataset.shape[-1]

1

I implement ResNet32 for a better performance, you can set the num_residual_blocks to define the number of layers.

In [33]:
batch_size = 16
patch_size = 5
num_residual_blocks = 5 # The number of layers = 1 + 2n + 2n + 2n + 1 = 6n + 2
weight_decay = 0.0002
bn_epsilon = 0.001

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Model Functions
    def create_variables(name, shape, initializer=tf.contrib.layers.xavier_initializer()):
        regularizer = tf.contrib.layers.l2_regularizer(scale=weight_decay)
        new_variables = tf.get_variable(name, shape=shape, initializer=initializer, regularizer=regularizer)
        return new_variables

    def batch_normalization_layer(input, dimension):
        mean, variance = tf.nn.moments(input, axes=[0, 1, 2])
        beta = tf.get_variable('beta', dimension, tf.float32, initializer=tf.constant_initializer(0.0, tf.float32))
        gamma = tf.get_variable('gamma', dimension, tf.float32, initializer=tf.constant_initializer(1.0, tf.float32))
        batch_normal = tf.nn.batch_normalization(input, mean, variance, beta, gamma, bn_epsilon)
        return batch_normal
  
    def conv_bn_relu_layer(input, filter_shape, stride):
        out_channel = filter_shape[-1]
        filter = create_variables(name='conv', shape=filter_shape)
        conv = tf.nn.conv2d(input, filter, strides = [1, stride, stride, 1], padding='SAME')
        bn = batch_normalization_layer(conv, out_channel)
        relu = tf.nn.relu(bn)
        return relu
    
    def bn_relu_conv_layer(input, filter_shape, stride):
        in_channel = input.get_shape().as_list()[-1]
        bn = batch_normalization_layer(input, in_channel)
        relu = tf.nn.relu(bn)
        filter = create_variables(name='conv', shape=filter_shape)
        conv = tf.nn.conv2d(relu, filter, strides=[1, stride, stride, 1], padding='SAME')
        return conv
    
    def residual_block(input, output_channel, first_block=False):
        input_channel = input.get_shape().as_list()[-1]
        
        if input_channel * 2 == output_channel:
            increase_dim = True
            stride = 2
        elif input_channel == output_channel:
            increase_dim = False
            stride = 1
        else:
            raise ValueError('Output channel cannot match the input channel')
        
        with tf.variable_scope('conv1_in_block'):
            if first_block:
                filter = create_variables(name='conv', shape=[3, 3, input_channel, output_channel])
                conv1 = tf.nn.conv2d(input, filter=filter, strides=[1, 1, 1, 1], padding='SAME')
            else:
                conv1 = conv_bn_relu_layer(input, [3, 3, input_channel, output_channel], stride)
        
        with tf.variable_scope('conv2_in_block'):
            conv2 = bn_relu_conv_layer(conv1, [3, 3, output_channel, output_channel], 1)
        
        if increase_dim == True:
            pooled_input = tf.nn.avg_pool(input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
            padded_input = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_channel//2, input_channel//2]])
        else:
            padded_input = input
        
        # Shortcut
        output = conv2 + padded_input
        return output
    
    def output_layer(input, num_labels):
        input_channel = input.get_shape().as_list()[-1]
        fc_w = create_variables(name='fc_weights', shape=[input_channel, num_labels],
                               initializer=tf.uniform_unit_scaling_initializer(factor=1.0))
        fc_b = create_variables(name='fc_biases', shape=[num_labels],initializer=tf.zeros_initializer())
        fc = tf.matmul(input, fc_w) + fc_b
        return fc
        
        
    # Model.
    def model(data, reuse):
        layers = []
        with tf.variable_scope('conv0', reuse=reuse):
            conv = conv_bn_relu_layer(data, [3, 3, 1, 16], 1)
            layers.append(conv)
        
        for i in range(num_residual_blocks):
            with tf.variable_scope('conv1_%d'%i, reuse=reuse):
                if i == 0:
                    conv = residual_block(layers[-1], 16, first_block=True)
                else:
                    conv = residual_block(layers[-1], 16)
                layers.append(conv)
        
        for i in range(num_residual_blocks):
            with tf.variable_scope('conv2_%d'%i, reuse=reuse):
                conv = residual_block(layers[-1], 32)
                layers.append(conv)
        
        for i in range(num_residual_blocks):
            with tf.variable_scope('conv3_%d'%i, reuse=reuse):
                conv = residual_block(layers[-1], 64)
                layers.append(conv)
            
        # FC 
        with tf.variable_scope('fc', reuse=reuse):
            in_channel = layers[-1].get_shape().as_list()[-1]
            bn = batch_normalization_layer(layers[-1], in_channel)
            relu = tf.nn.relu(bn)
            global_pool = tf.reduce_mean(relu, [1,2])
            output = output_layer(global_pool, num_labels)
            layers.append(output)
            
        return layers[-1]

    # Training computation.
    logits = model(tf_train_dataset, False)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, True))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, True))

In [34]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps): 
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[obffset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.375332
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 50: 1.690525
Minibatch accuracy: 56.2%
Validation accuracy: 50.0%
Minibatch loss at step 100: 1.131343
Minibatch accuracy: 87.5%
Validation accuracy: 69.1%
Minibatch loss at step 150: 0.992107
Minibatch accuracy: 68.8%
Validation accuracy: 74.7%
Minibatch loss at step 200: 0.587710
Minibatch accuracy: 93.8%
Validation accuracy: 76.0%
Minibatch loss at step 250: 0.860558
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 300: 1.052863
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 350: 1.189268
Minibatch accuracy: 62.5%
Validation accuracy: 81.4%
Minibatch loss at step 400: 0.724097
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 450: 0.371783
Minibatch accuracy: 93.8%
Validation accuracy: 83.2%
Minibatch loss at step 500: 0.757480
Minibatch accuracy: 68.8%
Validation accuracy: 83.2%
Min

It's better than the former convolutional network, but not the best. However, it's too expensive for me to optimize the hyper-parameters on my poor macbook pro, thus you can adjust the H-P by yourself.